In [1]:
#KGAT_746a8c48819a19cbaf8ca0048244831b

In [2]:
!pip install opendatasets
!pip install pandas

Скачиваем датасет

In [3]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/syahrulapriansyah2/myanimelist-2025")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: angrytea
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/syahrulapriansyah2/myanimelist-2025


100%|██████████| 9.57M/9.57M [00:00<00:00, 352MB/s]

Библиотеки

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, ClassifierMixin

Чтение датасета

In [5]:
pd.set_option("display.max_columns", None)
df = pd.read_csv("myanimelist-2025/mal_anime.csv")

Удаляем строки, не имеющие в Members значения  
Задаем условие успешности: тайтл успешный, если у него 75% и больше зрителей: 1 - аниме успешное (собрало много просмотров) 0 - не успешное  
Для признаков возьмем Source, Type, Released_Season, Episodes, Genres, Studios, Demographic  
Пропуски по эпизодам заменяем медианой, по категориальным признакам на Unknown, жанрам и студиям поставим пустые строки  
Создаем список жанров  
Ищем самые популярные жанры и делаем для них колонки  
Список студий делаем  
Ищем популярные студии, не учитывая 'add some'  
Задаем колонки  
Собираем матрицу X из фич и назначаем целевую переменную успешности y  
Делим данные на train и test  
Числовые и бинарные признаки будем масштабировать, а для категориальных используем one-hot encoding

In [6]:
df["Members"] = pd.to_numeric(df["Members"], errors="coerce")
df = df.dropna(subset=["Members"])
th = df["Members"].quantile(0.75)

df["hit"] = (df["Members"] >= th).astype(int)
df["Episodes"] = pd.to_numeric(df["Episodes"], errors="coerce")
m_episodes = df["Episodes"].median()
df["Episodes"] = df["Episodes"].fillna(m_episodes)

for col in ["Source", "Type", "Released_Season", "Demographic"]:
    df[col] = df[col].fillna("Unknown")

for col in ["Genres", "Studios"]: df[col] = df[col].fillna("")

df["Genres_l"] = df["Genres"].apply(lambda x: [i.strip() for i in str(x).split(",") if i.strip() != ""])
genre_count = Counter()

for g in df["Genres_l"]:
    genre_count.update(g)

top_g = [i for i, _ in genre_count.most_common(10)]

for g in top_g:
    col_name = f"genre_{g}"
    df[col_name] = df["Genres_l"].apply(lambda lst: int(g in lst))

g_cols = [f"genre_{g}" for g in top_g]
df["Studios_l"] = df["Studios"].apply(lambda x: [i.strip() for i in str(x).split(",") if i.strip() != ""])

studio_count = Counter()
for s in df["Studios_l"]:
  for st in s:
    clean_s = st.strip()
    if "add some" in clean_s.lower(): continue
    studio_count[clean_s] += 1

top_s = [i for i, _ in studio_count.most_common(10)]
for s in top_s:
    col_name = f"studio_{s}"
    df[col_name] = df["Studios_l"].apply(lambda lst: int(s in lst))

studio_cols = [f"studio_{s}" for s in top_s]

num = ["Episodes"] + g_cols + studio_cols
categ = ["Source", "Type", "Released_Season", "Demographic"]
feature = num + categ

X = df[feature]
y = df["hit"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
pred = ColumnTransformer(transformers=[
    ("num", StandardScaler(),num),
    ("categ", OneHotEncoder(handle_unknown="ignore"), categ)
])

Строю бейзлайн-модель решающего дерева для задачи классификации. Использую DecisionTreeClassifier со стандартными параметрами, обучаю на подготовленных признаках и считаю метрики качества на тесте: Accuracy, F1 и ROC-AUC, а также подробный отчёт по классам

Бейзлайновое решающее дерево даёт Accuracy ≈ 0.74 и F1 ≈ 0.37 по классу hit, при этом ROC-AUC около 0.60. Модель хорошо распознаёт неуспешные тайтлы (класс 0: precision 0.79, recall 0.88), но по успешным тайтлам (класс 1) сильно просаживается recall (0.31) и, как следствие, F1. То есть базовое дерево явно склонно к консервативным предсказаниям класса 0 и не очень уверенно выделяет действительно популярные аниме

In [7]:
tree_base = Pipeline(steps=[
    ("pred", pred),
    ("clf", DecisionTreeClassifier(random_state=42))
])

tree_base.fit(X_train, y_train)

y_pred_tree = tree_base.predict(X_test)
y_proba_tree = tree_base.predict_proba(X_test)[:, 1]

print("Бейзлайн")
print("Accuracy:", accuracy_score(y_test, y_pred_tree))
print("F1-score:", f1_score(y_test, y_pred_tree))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_tree))
print(classification_report(y_test, y_pred_tree))


Бейзлайн
Accuracy: 0.7352697095435685
F1-score: 0.3708086785009862
ROC-AUC: 0.6048510117122468
              precision    recall  f1-score   support

           0       0.79      0.88      0.83       903
           1       0.46      0.31      0.37       302

    accuracy                           0.74      1205
   macro avg       0.63      0.59      0.60      1205
weighted avg       0.71      0.74      0.72      1205



Первая гипотеза - улучшить качество дерева за счёт подбора гиперпараметров структуры. Чищу переобучение через max_depth, min_samples_split, min_samples_leaf и критерий разбиения (gini/entropy). Использую GridSearchCV с оптимизацией F1-меры по положительному классу hit

Подбор max_depth, min_samples_split и min_samples_leaf через GridSearchCV фактически вернул нам исходные настройки дерева (параметры совпали с дефолтом), и итоговые метрики полностью совпали с бейзлайном: Accuracy ≈ 0.74, F1 ≈ 0.37, ROC-AUC ≈ 0.60. Это говорит о том, что на текущем наборе признаков и при выбранной метрике (F1 по hit) автоматический подбор структуры дерева не даёт дополнительного выигрыша по качеству по сравнению с базовой конфигурацией

In [8]:
param_grid_tree1 = {
    "clf__criterion": ["gini", "entropy"],
    "clf__max_depth": [None, 5, 10, 15, 20],
    "clf__min_samples_split": [2, 10, 50],
    "clf__min_samples_leaf": [1, 5, 20]
}

tree_clf1 = Pipeline(steps=[
    ("pred", pred),
    ("clf", DecisionTreeClassifier(random_state=42))
])

grid_tree1 = GridSearchCV(
    estimator=tree_clf1,
    param_grid=param_grid_tree1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_tree1.fit(X_train, y_train)

print("Лучшие параметры:", grid_tree1.best_params_)
print("Лучший F1 на CV:", grid_tree1.best_score_)

best_tree1 = grid_tree1.best_estimator_
y_pred_tree1 = best_tree1.predict(X_test)
y_proba_tree1 = best_tree1.predict_proba(X_test)[:, 1]

print("\nГипотеза 1")
print("Accuracy:", accuracy_score(y_test, y_pred_tree1))
print("F1-score:", f1_score(y_test, y_pred_tree1))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_tree1))
print(classification_report(y_test, y_pred_tree1))


Лучшие параметры: {'clf__criterion': 'gini', 'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2}
Лучший F1 на CV: 0.36353307583885536

Гипотеза 1
Accuracy: 0.7352697095435685
F1-score: 0.3708086785009862
ROC-AUC: 0.6048510117122468
              precision    recall  f1-score   support

           0       0.79      0.88      0.83       903
           1       0.46      0.31      0.37       302

    accuracy                           0.74      1205
   macro avg       0.63      0.59      0.60      1205
weighted avg       0.71      0.74      0.72      1205



Вторая гипотеза - заменить исходное количество эпизодов на логарифм log(1 + Episodes), чтобы сгладить распределение этого признака. Строю новый набор признаков с log_Episodes, заново делю на train/test, настраиваю ColumnTransformer и снова подбираю гиперпараметры дерева по F1-мере

Добавление признака log_Episodes и повторный подбор гиперпараметров тоже не изменили итоговых метрик: Accuracy, F1 и ROC-AUC остались на уровне бейзлайна. То есть решающее дерево либо уже «видит» нужную информацию в сырых Episodes, либо влияние этого признака в целом невелико по сравнению с жанрами, студиями и категориальными фичами. С точки зрения качества модель ведёт себя идентично гипотезе 1, так что логарифмирование числа эпизодов в контексте дерева решений заметной пользы не даёт

In [9]:
df2 = df.copy()
df2["log_Episodes"] = np.log1p(df2["Episodes"])

num2 = ["log_Episodes"] + g_cols + studio_cols
categ = ["Source", "Type", "Released_Season", "Demographic"]
feature2 = num2 + categ

X2 = df2[feature2]
y2 = df2["hit"]

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2,
    y2,
    test_size=0.2,
    random_state=42,
    stratify=y2
)

pred2 = ColumnTransformer(transformers=[
    ("num2", StandardScaler(), num2),
    ("categ", OneHotEncoder(handle_unknown="ignore"), categ),
])

tree_clf2 = Pipeline(steps=[
    ("pred", pred2),
    ("clf", DecisionTreeClassifier(random_state=42))
])

param_grid_tree2 = param_grid_tree1

grid_tree2 = GridSearchCV(
    estimator=tree_clf2,
    param_grid=param_grid_tree2,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_tree2.fit(X2_train, y2_train)

print("Лучшие параметры:", grid_tree2.best_params_)
print("Лучший F1 на CV:", grid_tree2.best_score_)

best_tree2 = grid_tree2.best_estimator_
y_pred_tree2 = best_tree2.predict(X2_test)
y_proba_tree2 = best_tree2.predict_proba(X2_test)[:, 1]

print("\nDecision Tree + log(Episodes)")
print("Accuracy:", accuracy_score(y2_test, y_pred_tree2))
print("F1-score:", f1_score(y2_test, y_pred_tree2))
print("ROC-AUC:", roc_auc_score(y2_test, y_proba_tree2))
print(classification_report(y2_test, y_pred_tree2))


Лучшие параметры: {'clf__criterion': 'gini', 'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2}
Лучший F1 на CV: 0.3653184854459217

Decision Tree + log(Episodes)
Accuracy: 0.7352697095435685
F1-score: 0.3708086785009862
ROC-AUC: 0.6048510117122468
              precision    recall  f1-score   support

           0       0.79      0.88      0.83       903
           1       0.46      0.31      0.37       302

    accuracy                           0.74      1205
   macro avg       0.63      0.59      0.60      1205
weighted avg       0.71      0.74      0.72      1205



Третья гипотеза - увеличить размер one-hot-представления жанров и студий до топ-20 по частоте, чтобы дать дереву больше информативных бинарных признаков. Заново формирую признаки Episodes + top-20 жанров/студий, делю выборку, настраиваю препроцессинг и подбираю гиперпараметры дерева по F1

При использовании топ-20 жанров и студий вместо топ-10 F1 по классу hit даже немного просел (с ≈0.37 до ≈0.34), Accuracy тоже слегка упала (до ≈0.73), хотя ROC-AUC чуть вырос (≈0.62). Это можно интерпретировать так: модель стала чуть лучше ранжировать объекты по вероятности быть hit в целом, но на фиксованном пороге 0.5 стала ошибаться чаще в терминах F1 (особенно по редкому классу). Дополнительные бинарные признаки для более редких жанров/студий, по сути, добавляют шум и легко переобучают дерево, поэтому расширение до top-20 в нашей постановке задачи не улучшило, а скорее ухудшило качество по основной целевой метрике

In [10]:
top_s20 = [i for i, _ in studio_count.most_common(20)]
for s in top_s20:
    col_name = f"studio_{s}"
    df[col_name] = df["Studios_l"].apply(lambda lst: int(s in lst))
studio_cols20 = [f"studio_{s}" for s in top_s20]

top_g20 = [i for i, _ in genre_count.most_common(20)]
for g in top_g20:
    col_name = f"genre_{g}"
    df[col_name] = df["Genres_l"].apply(lambda lst: int(g in lst))
g_cols20 = [f"genre_{g}" for g in top_g20]

num3 = ["Episodes"] + g_cols20 + studio_cols20
categ3 = ["Source", "Type", "Released_Season", "Demographic"]
feature3 = num3 + categ3

X3 = df[feature3]
y3 = df["hit"]

X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3,
    y3,
    test_size=0.2,
    random_state=42,
    stratify=y3
)

pred3 = ColumnTransformer(transformers=[
    ("num", StandardScaler(), num3),
    ("categ", OneHotEncoder(handle_unknown="ignore"), categ3),
])

tree_clf3 = Pipeline(steps=[
    ("pred", pred3),
    ("clf", DecisionTreeClassifier(random_state=42))
])

param_grid_tree3 = param_grid_tree1

grid_tree3 = GridSearchCV(
    estimator=tree_clf3,
    param_grid=param_grid_tree3,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_tree3.fit(X3_train, y3_train)

print("Лучшие параметры:", grid_tree3.best_params_)
print("Лучший F1 на CV:", grid_tree3.best_score_)

best_tree3 = grid_tree3.best_estimator_
y_pred_tree3 = best_tree3.predict(X3_test)
y_proba_tree3 = best_tree3.predict_proba(X3_test)[:, 1]

print("\nDecision Tree + top-20 жанров/студий")
print("Accuracy:", accuracy_score(y3_test, y_pred_tree3))
print("F1-score:", f1_score(y3_test, y_pred_tree3))
print("ROC-AUC:", roc_auc_score(y3_test, y_proba_tree3))
print(classification_report(y3_test, y_pred_tree3))


Лучшие параметры: {'clf__criterion': 'entropy', 'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 10}
Лучший F1 на CV: 0.3878221423007429

Decision Tree + top-20 жанров/студий
Accuracy: 0.7311203319502074
F1-score: 0.3441295546558704
ROC-AUC: 0.617417658577369
              precision    recall  f1-score   support

           0       0.79      0.88      0.83       903
           1       0.44      0.28      0.34       302

    accuracy                           0.73      1205
   macro avg       0.61      0.58      0.59      1205
weighted avg       0.70      0.73      0.71      1205



Реализую собственный классификатор дерева решений  
Модель поддерживает бинарную классификацию и критерий качества Gini. При обучении в методе fit я сохраняю исходные классы, а затем рекурсивно строю дерево в виде вложенных словарей. На каждом узле считаю Gini-нечистоту, перебираю все признаки и возможные пороги (уникальные значения признака), считаю взвешенную Gini для левой и правой части и выбираю разбиение с минимальной нечистотой с учётом ограничений max_depth, min_samples_split и min_samples_leaf. Если дальше делить узел не имеет смысла (мало объектов, достигнута максимальная глубина или все объекты одного класса), создаётся лист, в котором хранится количество объектов каждого класса. В predict_proba я для каждого объекта спускаюсь по дереву (сравнивая значение признака с порогом) до листа и возвращаю нормализованные частоты классов как вероятности, а в predict беру класс с максимальной вероятностью и мапплю его обратно в исходные меткии

In [11]:
class MyDecisionTreeClassifier(BaseEstimator, ClassifierMixin):
    def __init__(
        self,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=None
    ):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.random_state = random_state

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        self.classes_ = np.unique(y)
        self.n_classes_ = self.classes_.shape[0]
        if self.n_classes_ != 2:
            raise ValueError("MyDecisionTreeClassifier сейчас поддерживает только бинарную классификацию")

        rng = np.random.RandomState(self.random_state)

        self.n_features_ = X.shape[1]
        self.tree_ = self._build_tree(X, y, depth=0, rng=rng)
        return self

    def _gini(self, y):
        m = y.shape[0]
        if m == 0:
            return 0.0
        _, counts = np.unique(y, return_counts=True)
        p = counts / m
        return 1.0 - np.sum(p ** 2)

    def _best_split(self, X, y):
        m, n = X.shape
        if m < self.min_samples_split:
            return None, None

        best_gini = None
        best_feature = None
        best_threshold = None

        current_gini = self._gini(y)

        for feature_idx in range(n):
            x_col = X[:, feature_idx]
            thresholds = np.unique(x_col)
            if thresholds.shape[0] == 1:
                continue

            for t in thresholds:
                left_mask = x_col <= t
                right_mask = x_col > t

                n_left = np.sum(left_mask)
                n_right = np.sum(right_mask)

                if n_left < self.min_samples_leaf or n_right < self.min_samples_leaf:
                    continue

                g_left = self._gini(y[left_mask])
                g_right = self._gini(y[right_mask])

                g_split = (n_left * g_left + n_right * g_right) / m

                if best_gini is None or g_split < best_gini:
                    best_gini = g_split
                    best_feature = feature_idx
                    best_threshold = t

        if best_gini is None or best_gini >= current_gini:
            return None, None

        return best_feature, best_threshold

    def _build_tree(self, X, y, depth, rng):
        node = {}
        num_samples = y.shape[0]
        num_labels = np.unique(y).shape[0]

        if num_labels == 1:
            node["type"] = "leaf"
            node["class_counts"] = self._class_counts(y)
            return node

        if self.max_depth is not None and depth >= self.max_depth:
            node["type"] = "leaf"
            node["class_counts"] = self._class_counts(y)
            return node

        if num_samples < self.min_samples_split:
            node["type"] = "leaf"
            node["class_counts"] = self._class_counts(y)
            return node

        feature_idx, threshold = self._best_split(X, y)

        if feature_idx is None:
            node["type"] = "leaf"
            node["class_counts"] = self._class_counts(y)
            return node

        left_mask = X[:, feature_idx] <= threshold
        right_mask = X[:, feature_idx] > threshold

        X_left, y_left = X[left_mask], y[left_mask]
        X_right, y_right = X[right_mask], y[right_mask]

        node["type"] = "internal"
        node["feature_idx"] = feature_idx
        node["threshold"] = threshold
        node["left"] = self._build_tree(X_left, y_left, depth + 1, rng)
        node["right"] = self._build_tree(X_right, y_right, depth + 1, rng)

        return node

    def _class_counts(self, y):
        counts = np.zeros(self.n_classes_, dtype=float)
        for idx, c in enumerate(self.classes_):
            counts[idx] = np.sum(y == c)
        return counts

    def _predict_one_proba(self, x, node):
        if node["type"] == "leaf":
            counts = node["class_counts"]
            total = np.sum(counts)
            if total == 0:
                return np.ones(self.n_classes_) / self.n_classes_
            return counts / total

        if x[node["feature_idx"]] <= node["threshold"]:
            return self._predict_one_proba(x, node["left"])
        else:
            return self._predict_one_proba(x, node["right"])

    def predict_proba(self, X):
        X = np.asarray(X)
        n_samples = X.shape[0]
        proba = np.zeros((n_samples, self.n_classes_), dtype=float)
        for i in range(n_samples):
            proba[i] = self._predict_one_proba(X[i], self.tree_)
        return proba

    def predict(self, X):
        proba = self.predict_proba(X)
        idx = np.argmax(proba, axis=1)
        return self.classes_[idx]

Строю бейзлайн на собственной реализации дерева решений  
Использую те же признаки и препроцессинг, что и для sklearn-дерева: StandardScaler для числовых признаков, one-hot для категориальных. Параметры дерева оставляю “по умолчанию” (неограниченная глубина, min_samples_split=2, min_samples_leaf=1)

Бейзлайновое дерево в моей реализации даёт Accuracy ≈ 0.74, F1 ≈ 0.37 и ROC-AUC ≈ 0.61. Картинка очень похожа на sklearn-дерево: класс 0 определяется хорошо (precision 0.79, recall 0.88), а успешные тайтлы (класс 1) по-прежнему ловим неидеально (recall 0.31, F1 около 0.37). Это значит, что базовое поведение моего дерева практически совпадает с библиотечной реализацией: модель уверенно узнаёт неуспешные аниме и осторожно прогнозирует класс hit

In [12]:
my_tree_base = Pipeline(steps=[
    ("pred", pred),
    ("clf", MyDecisionTreeClassifier(
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    ))
])

my_tree_base.fit(X_train, y_train)

y_pred_my_tree = my_tree_base.predict(X_test)
y_proba_my_tree = my_tree_base.predict_proba(X_test)[:, 1]

print("Бейзлайн")
print("Accuracy:", accuracy_score(y_test, y_pred_my_tree))
print("F1-score:", f1_score(y_test, y_pred_my_tree))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_my_tree))
print(classification_report(y_test, y_pred_my_tree))

Бейзлайн
Accuracy: 0.7394190871369295
F1-score: 0.3745019920318725
ROC-AUC: 0.6113910218330363
              precision    recall  f1-score   support

           0       0.79      0.88      0.84       903
           1       0.47      0.31      0.37       302

    accuracy                           0.74      1205
   macro avg       0.63      0.60      0.60      1205
weighted avg       0.71      0.74      0.72      1205



Гипотеза 1

GridSearchCV для MyDecisionTreeClassifier выбрал параметры, фактически совпадающие с бейзлайном, поэтому итоговые метрики полностью совпали с базовой моделью: Accuracy ≈ 0.74, F1 ≈ 0.37, ROC-AUC ≈ 0.61. Это означает, что внутри моей реализации та же логика роста дерева, и дополнительный перебор глубины/минимального размера узлов в заданной сетке не даёт прироста качества. По сути, гипотеза 1 лишь подтвердило, что настройка структуры дерева в таком виде не улучшает F1 по классу hit

In [13]:
param_grid_my_tree1 = {
    "clf__max_depth": [None, 5, 10, 15, 20],
    "clf__min_samples_split": [2, 10, 50],
    "clf__min_samples_leaf": [1, 5, 20]
}

my_tree_clf1 = Pipeline(steps=[
    ("pred", pred),
    ("clf", MyDecisionTreeClassifier(random_state=42))
])

grid_my_tree1 = GridSearchCV(
    estimator=my_tree_clf1,
    param_grid=param_grid_my_tree1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_my_tree1.fit(X_train, y_train)

print("Лучшие параметры:", grid_my_tree1.best_params_)
print("Лучший F1 на CV:", grid_my_tree1.best_score_)

best_my_tree1 = grid_my_tree1.best_estimator_

y_pred_my1 = best_my_tree1.predict(X_test)
y_proba_my1 = best_my_tree1.predict_proba(X_test)[:, 1]

print("\nГипотеза 1")
print("Accuracy:", accuracy_score(y_test, y_pred_my1))
print("F1-score:", f1_score(y_test, y_pred_my1))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_my1))
print(classification_report(y_test, y_pred_my1))


Лучшие параметры: {'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2}
Лучший F1 на CV: 0.36242371191736306

Гипотеза 1
Accuracy: 0.7394190871369295
F1-score: 0.3745019920318725
ROC-AUC: 0.6113910218330363
              precision    recall  f1-score   support

           0       0.79      0.88      0.84       903
           1       0.47      0.31      0.37       302

    accuracy                           0.74      1205
   macro avg       0.63      0.60      0.60      1205
weighted avg       0.71      0.74      0.72      1205



Гипотеза 2

Во второй гипотезе я логарифмировала число эпизодов и снова провела подбор гиперпараметров для MyDecisionTreeClassifier. Результаты снова ровно совпали с бейзлайном: те же лучшие параметры и те же значения Accuracy, F1 и ROC-AUC. Для дерева решений это ожидаемо: оно умеет само находить пороги по исходным значениям Episodes, и монотонное преобразование вида log не даёт ему дополнительной силы. В итоге логарифмирование эпизодов никак не меняет качество классификации в рамках используемой модели

In [14]:
my_tree_clf2 = Pipeline(steps=[
    ("pred", pred2),
    ("clf", MyDecisionTreeClassifier(random_state=42))
])

param_grid_my_tree2 = param_grid_my_tree1

grid_my_tree2 = GridSearchCV(
    estimator=my_tree_clf2,
    param_grid=param_grid_my_tree2,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_my_tree2.fit(X2_train, y2_train)

print("Лучшие параметры:", grid_my_tree2.best_params_)
print("Лучший F1 на CV:", grid_my_tree2.best_score_)

best_my_tree2 = grid_my_tree2.best_estimator_

y_pred_my2 = best_my_tree2.predict(X2_test)
y_proba_my2 = best_my_tree2.predict_proba(X2_test)[:, 1]

print("\nMyDecisionTreeClassifier + log(Episodes)")
print("Accuracy:", accuracy_score(y2_test, y_pred_my2))
print("F1-score:", f1_score(y2_test, y_pred_my2))
print("ROC-AUC:", roc_auc_score(y2_test, y_proba_my2))
print(classification_report(y2_test, y_pred_my2))


Лучшие параметры: {'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2}
Лучший F1 на CV: 0.36242371191736306

MyDecisionTreeClassifier + log(Episodes)
Accuracy: 0.7394190871369295
F1-score: 0.3745019920318725
ROC-AUC: 0.6113910218330363
              precision    recall  f1-score   support

           0       0.79      0.88      0.84       903
           1       0.47      0.31      0.37       302

    accuracy                           0.74      1205
   macro avg       0.63      0.60      0.60      1205
weighted avg       0.71      0.74      0.72      1205



Гипотеза 3

При добавлении top-20 жанров и студий Accuracy чуть выросла (≈0.742), ROC-AUC тоже подрос до ≈0.64, но F1 по классу hit заметно просел — примерно до 0.34. То есть дерево стало ещё сильнее фокусироваться на корректном предсказании основного класса 0 (неуспешные тайтлы, recall 0.90), но по редкому классу 1 начало ошибаться чаще (recall 0.26 и более низкий F1). Такое поведение похоже на лёгкое переобучение на дополнительные редкие бинарные признаки, которые добавляют шум, а не полезную структуру для нашей ключевой метрики

In [15]:
my_tree_clf3 = Pipeline(steps=[
    ("pred", pred3),
    ("clf", MyDecisionTreeClassifier(random_state=42))
])

param_grid_my_tree3 = param_grid_my_tree1

grid_my_tree3 = GridSearchCV(
    estimator=my_tree_clf3,
    param_grid=param_grid_my_tree3,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_my_tree3.fit(X3_train, y3_train)

print("Лучшие параметры:", grid_my_tree3.best_params_)
print("Лучший F1 на CV:", grid_my_tree3.best_score_)

best_my_tree3 = grid_my_tree3.best_estimator_

y_pred_my3 = best_my_tree3.predict(X3_test)
y_proba_my3 = best_my_tree3.predict_proba(X3_test)[:, 1]

print("\nMyDecisionTreeClassifier + top-20 жанров/студий")
print("Accuracy:", accuracy_score(y3_test, y_pred_my3))
print("F1-score:", f1_score(y3_test, y_pred_my3))
print("ROC-AUC:", roc_auc_score(y3_test, y_proba_my3))
print(classification_report(y3_test, y_pred_my3))


Лучшие параметры: {'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 10}
Лучший F1 на CV: 0.37975976662067296

MyDecisionTreeClassifier + top-20 жанров/студий
Accuracy: 0.7419087136929461
F1-score: 0.33970276008492567
ROC-AUC: 0.6441002398187059
              precision    recall  f1-score   support

           0       0.79      0.90      0.84       903
           1       0.47      0.26      0.34       302

    accuracy                           0.74      1205
   macro avg       0.63      0.58      0.59      1205
weighted avg       0.71      0.74      0.71      1205



В задаче предсказания успешности аниме мои эксперименты с решающим деревом показали, что и библиотечная, и собственная реализация дают очень похожие результаты: Accuracy около 0.73–0.74, F1 по классу hit на уровне ≈0.37 и ROC-AUC чуть выше 0.6. Тюнинг гиперпараметров дерева (глубина, минимальный размер узла и т.д.) и простые преобразования признаков (log(Episodes), расширение списка жанров и студий до top-20) либо практически не меняют качество, либо даже немного ухудшают F1 по целевому классу. При этом моя реализация дерева решений воспроизводит поведение sklearn-модели как по настройкам, которые выбирает GridSearchCV, так и по итоговым метрикам, поэтому её можно считать корректной. В целом решающее дерево на этом датасете даёт умеренное качество и сильно уступает по F1 более «гладким» моделям вроде логистической регрессии, а ключевую роль снова играет не столько сам алгоритм, сколько разумный выбор признаков и целевой метрики